In [ ]:
from google.colab import drive
drive.mount('/content/gdrive', force_remount=True)

In [ ]:
!pip install konlpy

In [ ]:
!pip install soykeyword

In [4]:
import pandas as pd
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import CountVectorizer
from konlpy.tag import Okt
from tqdm import tqdm
import re
from soykeyword.proportion import CorpusbasedKeywordExtractor

In [ ]:
df = pd.read_excel('/content/gdrive/MyDrive/싼데비스탄/비버웍스_샘플데이터_open.xlsx', sheet_name=None)
dirty = pd.read_excel('/content/gdrive/MyDrive/싼데비스탄/해커톤참고자료(dirty).xlsx')

In [6]:
# local
df = pd.read_excel('/Users/badro97/NLP/NLP_Practice/비버웍스_샘플데이터_open.xlsx', sheet_name=None)
dirty = pd.read_excel('/Users/badro97/NLP/NLP_Practice/해커톤참고자료(dirty).xlsx')

In [7]:
orderQueen = df['오더퀸']
unos = df['우노스']

# 전처리
### 모든 데이터에 적용 가능해야 함
- 동일한 데이터프레임 Form으로 변경
- 취사선택 기능 (제거할 특수문자 입력)
- 동일한 기준 설정 (영어, 단위 제거)
- 오탈자 교정 (자모 단위)
- 띄어쓰기 교정 (soynlp - soyspacing)



**1/16 아이디어**

doc2vec or word2vec kmeans(elbow method)가 줄어들때 까지

태그로 뽑은 단어 제목에 추가(빈도 수 기준(뒷 단어 기준))

스캐터플롯으로 밀집도 변화 관찰

마지막에 메뉴명 최종 전처리(자모 교정, 토큰화, pororo, 외래어 표기법)

In [8]:
## 데이터프레임 form 통일
def form(df, brand, title, category=''):
  data = pd.DataFrame()
  data['brand'] = df[brand]
  data['title'] = df[title]
  data['category'] = df[category] if category else np.nan
  return data

In [9]:
o = form(orderQueen, '브랜드명', 'MENU_FULL_NM')
u = form(unos, '매장명', '메뉴명', '카테고리명')
d = form(dirty, '상점명', '상품명', '카테고리명')

df_list = [o, u, d]
df = pd.concat(df_list, ignore_index=True)
df

,brand,title,category
0,라쿤카페,르완다 마쿠바 풀리,NaN
1,라쿤카페,과테말라 세리토스,NaN
2,라쿤카페,에티오피아 모모라 구지 G1,NaN
3,라쿤카페,브라질 만치케이라 드 미나스,NaN
4,라쿤카페,오늘의 커피,NaN
...,...,...,...
1012,어른김밥,---------Take out---------,어른 김밥
1013,어른김밥,--------- 전화 결제X ---------,어른 김밥
1014,(주)담청에프엔비,(1입)TBL-벤토나이트 카본 모래_울트라오더 씰 12KG,간식
1015,커피파운드,[S&H]RWANDA KARIMA\n살구 청을 달콤하게 우려낸 듯 깔끔한 커피,필터커피


In [10]:
df[df['category'].isnull()]

,brand,title,category
0,라쿤카페,르완다 마쿠바 풀리,NaN
1,라쿤카페,과테말라 세리토스,NaN
2,라쿤카페,에티오피아 모모라 구지 G1,NaN
3,라쿤카페,브라질 만치케이라 드 미나스,NaN
4,라쿤카페,오늘의 커피,NaN
...,...,...,...
507,포케올데이_시흥은계점,정기구독 15+2,NaN
508,포케올데이_시흥은계점,텀블러,NaN
560,포케올데이_시흥은계점,메인토핑추가금,NaN
561,포케올데이_시흥은계점,메인토핑추가,NaN


In [11]:
df['category'].unique()

array([nan, 'Poke 포케', 'Set 세트', '대표메뉴', 'Wrap 랩', '두부볼', 'Soup 수프',
       '배달비', 'Drink 음료', '여름메뉴', '포장여부', '계절메뉴', '정기구독', '배배꼬 세트',
       '크림도넛', '음료', '베이커리', '시그니처 메뉴', 'COFFEE', 'NON COFFEE', '스무디볼',
       '와플&크로플', '케이크', '베이글&마카롱', '보틀&파우치', '커피', '라떼(non-coffee)',
       '티&에이드&주스', '스무디&프라페', '빙수', '핫도그', 'DEFAULT', '에이드&주스&스무디',
       '테스트계정', '커피/라떼', '세트메뉴', '이호당 DONUTS', '아이스크림 시럽 컵', '발룬티어스',
       '곁들임 메뉴', '포장', '1인 세트 (약 15% 할인)', 'EVENT',
       'JIMIN DEBUT 9TH ANNIVERSARY ', 'Coffee', 'Restaurant', '한입™사이드',
       'TEA', '오전', '미소카츠', '넌커피', '*넌커피', 'TAKE OUT', '肉松·로우쏭', '??·화과자',
       '블로거', 'DESSERT', '용품', '어른 김밥', '간식', '필터커피'], dtype=object)

In [12]:
df.columns

Index(['brand', 'title', 'category'], dtype='object')

In [13]:
## 토큰들 문자열로 결합
def tokenToString(token_list):
    res = []
    for token in token_list:
      s = ' '.join(token)
      res.append(s)
    return res

In [14]:
## []안에 문자가 있다? (브랜드명), ()안에 한글이 있다? or 숫자뒤에 한글/영어가 온다? (단위)
pat = re.compile("\[[^)]*\]|\([가-힣]\)|[0-9]+[a-zA-Z가-힣]*") 
## 특수문자 제외
sp = re.compile("[^가-힣a-zA-Z\s]")

## 지우는 전처리
def remove_eda(df, col):
  token_list = []
  for t in tqdm(df[col]):
    if t:
      t1 = pat.sub("", str(t))
      text = sp.sub(" ", t1).lower()
      res = text.split()
      token_list.append(res)
    else:
      token_list.append(t)
  result = tokenToString(token_list)
  df[col] = result
  return df

In [ ]:
# ## 지우는 전처리
# def remove_eda(df, col):
#   token_list = remove_special(df, col)
#   tok = []
#   for text in tqdm(token_list):
#     li = []
#     for i in text:
#       if any(temp.isdigit() for temp in i) == False: # 숫자와 붙어서 단어로 묶인 것들은 포함하지 않는다.
#         li.append(i)
#     tok.append(li)
#   result = tokenToString(tok)
#   df[col] = result
#   return df

In [15]:
for col in df.columns:
  df2 = remove_eda(df, col)

100%|██████████| 1017/1017 [00:00<00:00, 584409.81it/s]


In [16]:
df2

,brand,title,category
0,라쿤카페,르완다 마쿠바 풀리,nan
1,라쿤카페,과테말라 세리토스,nan
2,라쿤카페,에티오피아 모모라 구지 g,nan
3,라쿤카페,브라질 만치케이라 드 미나스,nan
4,라쿤카페,오늘의 커피,nan
...,...,...,...
1012,어른김밥,take out,어른 김밥
1013,어른김밥,전화 결제x,어른 김밥
1014,담청에프엔비,tbl 벤토나이트 카본 모래 울트라오더 씰,간식
1015,커피파운드,rwanda karima 살구 청을 달콤하게 우려낸 듯 깔끔한 커피,필터커피


In [17]:
df2['category'].unique()

array(['nan', 'poke 포케', 'set 세트', '대표메뉴', 'wrap 랩', '두부볼', 'soup 수프',
       '배달비', 'drink 음료', '여름메뉴', '포장여부', '계절메뉴', '정기구독', '배배꼬 세트',
       '크림도넛', '음료', '베이커리', '시그니처 메뉴', 'coffee', 'non coffee', '스무디볼',
       '와플 크로플', '케이크', '베이글 마카롱', '보틀 파우치', '커피', '라떼 non coffee',
       '티 에이드 주스', '스무디 프라페', '빙수', '핫도그', 'default', '에이드 주스 스무디',
       '테스트계정', '커피 라떼', '세트메뉴', '이호당 donuts', '아이스크림 시럽 컵', '발룬티어스',
       '곁들임 메뉴', '포장', '세트 약 할인', 'event', 'jimin debut anniversary',
       'restaurant', '한입 사이드', 'tea', '오전', '미소카츠', '넌커피', 'take out',
       '로우쏭', '화과자', '블로거', 'dessert', '용품', '어른 김밥', '간식', '필터커피'],
      dtype=object)

In [16]:
# 카테고리명split 한 값이 두개이상 영어가 섞여있다면 한글 선택
# 두개 이상이고 모두 한글이라면 메뉴와 가장 가까운 값 선택 (pororo 제로샷 분류)
# 모두 영어일 경우 따로 모아서 수동으로 변경


In [18]:
df2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1017 entries, 0 to 1016
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   brand     1017 non-null   object
 1   title     1017 non-null   object
 2   category  1017 non-null   object
dtypes: object(3)
memory usage: 24.0+ KB


In [18]:
## 키워드 뽑기 (빈도수)
from soykeyword.proportion import CorpusbasedKeywordExtractor
corpusbased_extractor = CorpusbasedKeywordExtractor(
    min_tf=2,
    min_df=1,
    tokenize=lambda x:x.strip().split(),
    verbose=True
)

# docs: list of str like
corpusbased_extractor.train(df2['title'])

training was done 854 terms, 1017 docs, memory = 0.043 Gb


In [19]:
keywords = corpusbased_extractor.extract_from_word(
    '카모마일',
    min_score=0.5,
    min_frequency=0
)

In [20]:
keywords

[KeywordScore(word='스위트', frequency=1, score=1.0),
 KeywordScore(word='카모마일', frequency=1, score=1.0)]

## OrderQueen

In [21]:
orderQueen.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 499 entries, 0 to 498
Data columns (total 9 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   브랜드코드         499 non-null    object
 1   브랜드명          499 non-null    object
 2   점포번호          499 non-null    int64 
 3   점포명           499 non-null    object
 4   사업자번호         499 non-null    object
 5   MENU_CD       499 non-null    object
 6   MENU_FULL_NM  499 non-null    object
 7   MENU_NM       499 non-null    object
 8   판매가격          499 non-null    int64 
dtypes: int64(2), object(7)
memory usage: 35.2+ KB


In [22]:
orderQueen.duplicated().sum()

0

In [23]:
orderQueen.head()

,브랜드코드,브랜드명,점포번호,점포명,사업자번호,MENU_CD,MENU_FULL_NM,MENU_NM,판매가격
0,rakun,라쿤카페,1001,라쿤라떼17,349-02-01908,B0000109,르완다 마쿠바 풀리,르완다 마쿠바 풀리,5500
1,rakun,라쿤카페,1001,라쿤라떼17,349-02-01908,B0000110,과테말라 세리토스,과테말라 세리토스,4500
2,rakun,라쿤카페,1001,라쿤라떼17,349-02-01908,B0000111,에티오피아 모모라 구지 G1,에티오피아 모모라 구지 G,5500
3,rakun,라쿤카페,1001,라쿤라떼17,349-02-01908,B0000112,브라질 만치케이라 드 미나스,브라질 만치케이라 드 미나,5500
4,rakun,라쿤카페,1001,라쿤라떼17,349-02-01908,B0000113,오늘의 커피,오늘의 커피,5000


In [24]:
orderQueen.tail()

,브랜드코드,브랜드명,점포번호,점포명,사업자번호,MENU_CD,MENU_FULL_NM,MENU_NM,판매가격
494,coffeemama,커피마마퀸,1024,커피마마퀸 조암점,847-87-00943,B0000150,건포도 스콘,건포도,0
495,coffeemama,커피마마퀸,1024,커피마마퀸 조암점,847-87-00943,B0000151,호두 스콘,호두,0
496,coffeemama,커피마마퀸,1024,커피마마퀸 조암점,847-87-00943,B0000152,바닐라 젤라소프트,바닐라,0
497,coffeemama,커피마마퀸,1024,커피마마퀸 조암점,847-87-00943,B0000153,바닐라&초코 젤라소프트,바닐라&초코,0
498,coffeemama,커피마마퀸,1024,커피마마퀸 조암점,847-87-00943,B0000154,망고 젤라소프트,망고,0


# Unos

In [25]:
unos.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 468 entries, 0 to 467
Data columns (total 6 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   affiliate_id  468 non-null    object
 1   매장명           468 non-null    object
 2   메뉴코드          468 non-null    int64 
 3   카테고리명         455 non-null    object
 4   메뉴명           468 non-null    object
 5   가격            468 non-null    int64 
dtypes: int64(2), object(4)
memory usage: 22.1+ KB


In [26]:
unos[unos['카테고리명'].isnull()]

,affiliate_id,매장명,메뉴코드,카테고리명,메뉴명,가격
0,pokeallday_eg,포케올데이_시흥은계점,1295195,NaN,메인토핑추가,2000
1,pokeallday_eg,포케올데이_시흥은계점,1295196,NaN,아보카도추가,2000
2,pokeallday_eg,포케올데이_시흥은계점,1295197,NaN,또띠아빵추가,1500
3,pokeallday_eg,포케올데이_시흥은계점,1295198,NaN,현미밥추가,1000
4,pokeallday_eg,포케올데이_시흥은계점,1295199,NaN,메밀면추가,1000
5,pokeallday_eg,포케올데이_시흥은계점,1295200,NaN,캔들판매,12900
6,pokeallday_eg,포케올데이_시흥은계점,1295201,NaN,멤버쉽 가입,20000
7,pokeallday_eg,포케올데이_시흥은계점,1295202,NaN,정기구독 10+1,109000
8,pokeallday_eg,포케올데이_시흥은계점,1295203,NaN,정기구독 15+2,163500
9,pokeallday_eg,포케올데이_시흥은계점,1295204,NaN,텀블러,10900


In [27]:
unos['메뉴코드'].duplicated().sum()

0

In [28]:
unos.head()

,affiliate_id,매장명,메뉴코드,카테고리명,메뉴명,가격
0,pokeallday_eg,포케올데이_시흥은계점,1295195,NaN,메인토핑추가,2000
1,pokeallday_eg,포케올데이_시흥은계점,1295196,NaN,아보카도추가,2000
2,pokeallday_eg,포케올데이_시흥은계점,1295197,NaN,또띠아빵추가,1500
3,pokeallday_eg,포케올데이_시흥은계점,1295198,NaN,현미밥추가,1000
4,pokeallday_eg,포케올데이_시흥은계점,1295199,NaN,메밀면추가,1000


In [29]:
unos.tail()

,affiliate_id,매장명,메뉴코드,카테고리명,메뉴명,가격
463,ogasalad_jhs,오가샐러드_중화산점,1671771,테스트계정,1800-4520에2번,2000
464,ogasalad_jhs,오가샐러드_중화산점,1671772,테스트계정,테스트결제는가능합니다,100
465,ogasalad_jhs,오가샐러드_중화산점,1671773,에이드&주스&스무디,(ICE)레몬에이드,5000
466,ogasalad_jhs,오가샐러드_중화산점,1671774,에이드&주스&스무디,된장스무디,80000
467,ogasalad_jhs,오가샐러드_중화산점,1673323,DEFAULT,테스트,1000


# Dirty Sample

In [30]:
dirty.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50 entries, 0 to 49
Data columns (total 8 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   상점코드    50 non-null     object 
 1   상점명     50 non-null     object 
 2   카테고리코드  50 non-null     int64  
 3   카테고리명   50 non-null     object 
 4   상품코드    50 non-null     int64  
 5   상품명     50 non-null     object 
 6   가격      49 non-null     float64
 7   사용여부    49 non-null     object 
dtypes: float64(1), int64(2), object(5)
memory usage: 3.2+ KB


In [31]:
dirty

,상점코드,상점명,카테고리코드,카테고리명,상품코드,상품명,가격,사용여부
0,woki_sandwich,우키샌드위치_송정점,32112,커피/라떼,1590070,(?˙?˙?) 2배용량 사이즈업 아이스 카페라떼,8600.0,Y
1,wapi_mia,와피_미아역점,59171,세트메뉴,1021509,A.대만샌드위치+아메리카노,4000.0,Y
2,twotigerscafe,이호당,85761,이호당 DONUTS,1514249,(DEEP)_누텔라 초코 도넛,3300.0,Y
3,twotigerscafe,이호당,85761,이호당 DONUTS,1514247,(FRESH)_우유 생크림 도넛,2800.0,Y
4,twotigerscafe,이호당,85761,이호당 DONUTS,1584497,(NEW)_얼그레이 커스터드 도넛,3300.0,Y
5,twotigerscafe,이호당,85761,이호당 DONUTS,1514257,(BOX)_프리미엄 도넛박스,18500.0,Y
6,theverywa_kor,더베리와플 안암점,7311,아이스크림 시럽 컵,168860,90.요거트+블루베리,3700.0,N
7,soulbutton,소울버튼,65285,발룬티어스,1125124,Radio 틀고 Let me go! + 봉사자들사랑해카롱♥,9000.0,N
8,songbin_kamameshi,가마메시야 송빈,81297,곁들임 메뉴,1475000,부타노 쇼가야키 (豚の生姜?き)\n최상급 마블링 목살구이. 은은한 생강향이 일품.,8000.0,Y
9,schoolfood_sm,스쿨푸드_서울성모병원점,43894,포장,902162,(포)시즌4) 매니아냉면 + 군만두(3pcs),10000.0,Y
